In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns" ,100)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv", index_col="Id")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv", index_col="Id")
df = pd.concat([train.iloc[:,:-1], test], axis=0)
y = train["SalePrice"]

In [ ]:
sns.histplot(y)

In [ ]:
cat_cols = [col for col in df.columns[:-1] if df[col].dtype=="object"]
num_cols = [col for col in df.columns[:-1] if df[col].dtype!="object"]

In [ ]:
missing = ["{}:    ".format(col)+str((df[col].isnull().sum())/df.shape[0]) for col in df.columns if df[col].isnull().sum()>0]
missing

In [ ]:
df.loc[ df["MasVnrType"].isnull() & df["MasVnrArea"].isnull(), ["MasVnrType", "MasVnrArea"]] = ["NA", 0]

df.loc[ df["TotalBsmtSF"]==0, ["BsmtQual", "BsmtCond", "BsmtExposure"]] = ["NA", "NA", "NA"]
df.loc[ df["TotalBsmtSF"]==0, ["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2"]] = ["NA", 0, "NA"]
df.loc[ df["TotalBsmtSF"]==0, ["BsmtFinSF2", "BsmtUnfSF"]] = [0, 0,]

df.loc[ df["TotalBsmtSF"]==0, ["BsmtQual", "BsmtCond", "BsmtExposure"]] = ["NA","NA","NA"]
df.loc[ df["TotalBsmtSF"]==0, ["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2"]] = ["NA",0,"NA"]
df.loc[ df["BsmtQual"].isnull() & df["BsmtCond"].isnull(), ["BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2"]] = ["NA","NA",0,"NA"]
df.loc[ df["BsmtQual"].isnull() & df["BsmtCond"].isnull(), ["BsmtQual", "BsmtCond"]] = ["NA","NA"]
df.loc[ ( df["BsmtQual"]=="NA") & (df["BsmtCond"]=="NA"), ["BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF"]] = [0,0,0]
df.loc[ ( df["BsmtQual"]=="NA") & (df["BsmtCond"]=="NA"), ["BsmtFullBath", "BsmtHalfBath"]] = [0,0]

df.loc[ df["GarageArea"]==0, ["GarageType", "GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond"]] = ["NA", 0, "NA", "NA","NA"]
df.loc[ df["GarageCars"].isnull() , ["GarageType", "GarageYrBlt", "GarageFinish", "GarageArea", "GarageQual", "GarageCond"]] = ["NA", 0, "NA",0,"NA","NA"]

df["PoolQC"].fillna("NA", inplace=True)
#df["Alley"].fillna("NA", inplace=True)
df["Utilities"].fillna("NA", inplace=True)
#df["Fence"].fillna("NA", inplace=True)
df["MiscFeature"].fillna("NA", inplace=True)
df["FireplaceQu"].fillna("NA", inplace=True)

df.BsmtExposure.loc[df.BsmtExposure=="No"] = "NA"

In [ ]:
lot_shape = { "Reg":1, "IR1":2, 'IR2':3,'IR3':4}
land_contour = {'Low':1,'HLS':2,'Bnk':3,'Lvl':4}
lot_config = {"Inside":1,"Corner":2,"CulDSac":3, "FR2":4, "FR3":5  }
land_slope = {"Gtl":1, "Mod":2, "Sev":3}
bsmt_exposure = {"NA":0, "Mn":1, "Av":2, "Gd":3}
bsmt_type = { "NA":0, "Unf":1, 'LwQ':2,'Rec':3, 'BLQ':4, 'ALQ':5, "GLQ":6}
garage_finish = {"NA":0, "Unf":1, "RFn":2, "Fin":3}
paved_drive = {"N":0, "P":1, "Y":2}
fence = {"NA":0, "MnWw":1, "GdWo":2, "MnPrv":3, "GdPrv":4}
street = {"Grvl":1, "Pave":2}
qual = { "NA":0, "Po":1, 'Fa':2,'TA':3, 'Gd':4, 'Ex':5}

df["LotShape"] = df["LotShape"].map(lot_shape)
df["LandContour"] = df["LandContour"].map(land_contour)
df["LotConfig"] = df["LotConfig"].map(lot_config)
df["LandSlope"] = df["LandSlope"].map(land_slope)
df["BsmtExposure"] = df["BsmtExposure"].map(bsmt_exposure)
df["BsmtFinType1"] = df["BsmtFinType1"].map(bsmt_type)
df["BsmtFinType2"]= df["BsmtFinType2"].map(bsmt_type)
df["GarageFinish"] = df["GarageFinish"].map(garage_finish)
df["PavedDrive"] = df["PavedDrive"].map(paved_drive)
#df["Fence"] = df["Fence"].map(fence)
df["Street"] = df["Street"].map(street)

features =[ "ExterQual", "BsmtCond","BsmtQual","PoolQC", "GarageCond", "GarageQual", "FireplaceQu" ,"KitchenQual", "HeatingQC", "ExterCond"]
for feat in features:
    df[feat] = df[feat].map(qual)

In [ ]:
df['MSZoning'] =  df['MSZoning'].replace({'C (all)': 'C'})
df["Exterior1st"] = df["Exterior1st"].replace({"Wd Sdng":"WdSdng"})
df["Exterior2nd"] = df["Exterior2nd"].replace({"Brk Cmn":"BrkComm","Wd Shng": "WdShing","Wd Sdng":"WdSdng" })
df["GarageYrBlt"] = df["GarageYrBlt"].where(df.GarageYrBlt <= 2010, df.YearBuilt)
df['Season_Sold'] = [1 if (i>3) & (i<9) else 0 for i in df['MoSold']]

In [ ]:
filtr = lambda row: int(row["OpenPorchSF"]>0) + int(row["EnclosedPorch"]>0) + int(row["3SsnPorch"]>0) + int(row["ScreenPorch"]>0)
df['NumPorch'] = df.apply(filtr,axis=1)

filtr1 = lambda row: int(row["1stFlrSF"]>0) + int(row["2ndFlrSF"]>0)
df['FloorNum'] = df.apply(filtr1,axis=1)

In [ ]:
df["LivLotRatio"] = df["GrLivArea"] / df["LotArea"]
df["Spaciousness"] = (df["1stFlrSF"]+df["2ndFlrSF"]) / df["TotRmsAbvGrd"]
df["TotalOutsideSF"] = df["WoodDeckSF"] + df["OpenPorchSF"] + df["EnclosedPorch"] + df["3SsnPorch"] + df["ScreenPorch"]
df["LowQuality"] = df["LowQualFinSF"]/ df["GrLivArea"]
df['TotalLot'] = df['LotFrontage'] + df['LotArea']
df['TotalBsmtFin'] = df['BsmtFinSF1'] + df['BsmtFinSF2']
df['TotalSF'] = df['TotalBsmtSF'] + df['2ndFlrSF'] + df['1stFlrSF']
df["BasementFin"] = (df["TotalBsmtSF"] - df["BsmtUnfSF"]) / df["TotalBsmtSF"]
df["TotalBathUp"] = df["FullBath"] + df["HalfBath"]*.5
df["BasementBath"] = df["BsmtFullBath"] + df["BsmtHalfBath"]*.5
df["Renovation"] = df["YrSold"] - df["YearRemodAdd"]
df["Renovation"] = df['Renovation'].where(df['YearBuilt']!=df["YearRemodAdd"], 0)
df["Age"] = df["YrSold"] - df["YearBuilt"]
df['TotalBath'] = df['FullBath'] + df['HalfBath'] * 0.5 + df['BsmtFullBath'] + df['BsmtHalfBath'] * 0.5
df['TotalPorch'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF']

In [ ]:
df.drop(["Alley","Fence","YearRemodAdd", "YearBuilt", "MSSubClass", "Street", "Utilities", 'PoolQC', 'MiscFeature', 
         'Fence', 'MoSold', 'YrSold',"PoolQC","FireplaceQu" ], axis=1, inplace=True)

In [ ]:
cat_cols_low = [col for col in df.columns[:-1] if df[col].dtype=="object" and df[col].nunique()<10]
cat_cols_high = [col for col in df.columns[:-1] if df[col].dtype=="object" and df[col].nunique()>10]
cat_cols = [col for col in df.columns[:-1] if df[col].dtype=="object"]
num_cols = [col for col in df.columns[:-1] if df[col].dtype!="object"]

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer

imputer_cat = SimpleImputer(strategy="most_frequent")
df_cat = pd.DataFrame(imputer_cat.fit_transform(df[cat_cols]), columns=cat_cols)
    
imputer = KNNImputer(n_neighbors=5,)
df_num = pd.DataFrame(imputer.fit_transform(df[num_cols]), columns=num_cols)

In [ ]:
sum(df_num.isnull().sum()), sum(df_cat.isnull().sum())

In [ ]:
plt.figure(figsize=(20,12))
df_num.corr().iloc[:-1,-1].sort_values().plot(kind="bar")

In [ ]:
fig, ax = plt.subplots(figsize=(25,20), tight_layout=True,)
corr = df_num.corr()
mask = np.triu(corr)
sns.heatmap(corr, annot=True, mask=mask, cmap = "coolwarm", ax = ax)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(df_num.values, i) for i in range(df_num.shape[1])]
vif["features"] = df_num.columns
vif.sort_values(by="VIF", ascending=False)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
encoder_ohe = OneHotEncoder(handle_unknown = "ignore", sparse=False)
df_cat_low = pd.DataFrame(encoder_ohe.fit_transform(df_cat[cat_cols_low]), index=df_cat[cat_cols_low].index)
df_cat_low.columns= encoder_ohe.get_feature_names()

encoder_oe = OrdinalEncoder()
df_cat_high = pd.DataFrame(encoder_oe.fit_transform(df_cat[cat_cols_high]), index=df_cat[cat_cols_high].index, 
                           columns = cat_cols_high )

In [ ]:
df_new = pd.concat([df_cat_low,df_cat_high, df_num], axis=1)

In [ ]:
df_new.shape

In [ ]:
sum(df_new.isnull().sum())

In [ ]:
df_train = df_new.iloc[:1460,]
df_test = df_new.iloc[1460:,]

In [ ]:
from sklearn.feature_selection import mutual_info_regression

scores =  mutual_info_regression(df_train, y)
cols = [x for i, x in enumerate(df_train.columns) if scores[i] != 0]

In [ ]:
len(cols)

In [ ]:
df_train = df_train[cols]
df_test = df_test[cols]

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

In [ ]:
#X_train, X_valid, y_train, y_valid = train_test_split(df_train, y, random_state=0, test_size=0.2)

In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, ElasticNetCV, LassoCV, Ridge,BayesianRidge
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
cv= KFold(n_splits=5, shuffle=True, random_state=0)

models = {
    #'LR' : LinearRegression(),
    'LGBMR' : LGBMRegressor(),
    "histGrad":HistGradientBoostingRegressor(),
    'CatBoost' : CatBoostRegressor(loss_function='MAE', random_state=0, iterations=1000,),
    'GBR' : GradientBoostingRegressor(loss='huber'),
    'XGB' : XGBRegressor(objective='reg:squarederror'),
    'RF': RandomForestRegressor(n_estimators=100, random_state=0),

}

scores = pd.DataFrame( columns=["model", "mean_mae", "std_mae", "r2"])

for key,value in models.items():
    mae_score = -cross_val_score(estimator=value, X=df_train, y=y, cv=cv, scoring="neg_mean_absolute_error", verbose=0 )
    r_score = cross_val_score(estimator=value, X=df_train, y=y, cv=cv, scoring="r2", verbose=0 )
    
    scores = scores.append(pd.DataFrame([[key, mae_score.mean(), mae_score.std(), r_score.mean()]], columns=scores.columns)) 

In [ ]:
scores.sort_values('mean_mae', ascending=True)

In [ ]:
import optuna
import gc

def objective(trial):
    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    cv_scores = np.empty(5)
    
    for fold, (train_idx, test_idx) in enumerate(kfold.split(df_train, y)):
        X_train, X_valid = df_train.iloc[train_idx], df_train.iloc[test_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[test_idx]
        params = {
            'bootstrap_type': 'Bernoulli',
            #'task_type' : 'GPU',
            #'devices' : '0' ,
            "random_state":0,

            'n_estimators':3000,
            'objective': 'MAE',
            'od_wait':trial.suggest_int('od_wait', 500,2300),
            #'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
            "learning_rate" : trial.suggest_float("learning_rate", 1e-2, 0.25, log=True),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100.0),
            'random_strength': trial.suggest_uniform('random_strength',10,50),
            'depth': trial.suggest_int('depth',1,10),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
            "subsample" : trial.suggest_float("subsample", 0.1, 1.0),
            #"colsample_bylevel" : trial.suggest_float("colsample_bylevel", 0.1, 1.0),
            'border_count':trial.suggest_int("border_count", 1,255),
            "verbose":0


        }
            
            #params['grow_policy'] = 'Depthwise'

        #if params['bootstrap_type'] == 'Bayesian':
        #    params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
        #elif params['bootstrap_type'] == 'Bernoulli':
          #  params['subsample'] = trial.suggest_float('subsample', 0.1, 1)

    model = CatBoostRegressor(**params, eval_metric='MAE')
    model.fit( X_train, y_train, eval_set=[(X_valid,y_valid)],
              early_stopping_rounds=50, use_best_model=True, verbose=0)
            
    # validation prediction
    pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, pred)

    del X_train, y_train, X_valid, y_valid, pred
    gc.collect()
    return mae

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, 
               #n_trials=100,
               #n_jobs = multiprocessing.cpu_count(),
              timeout=1*60*60
              )

In [ ]:
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
cat_params= {
            'bootstrap_type': 'Bernoulli',
            "random_state":0,
            'n_estimators':3000,

 'objective': 'MAE',
 'od_wait': 1781,
  'learning_rate': 0.01929597138080446, 
  'reg_lambda': 0.00019373164739481443, 
  'random_strength': 36.02316299599894, 
  'depth': 3, 
  'min_data_in_leaf': 27, 
  'leaf_estimation_iterations': 3, 
  'subsample': 0.15385862961015184, 
  'border_count': 88}
#Best score: 12105.226409393237


In [ ]:
cat_params_0= {
    #'task_type' : 'GPU',
    #'devices' : '0' ,
    "random_state":0,
'iterations': 796,
 'objective': 'MAE', 
 'od_wait': 1731,
 'bootstrap_type': 'MVS',
 'learning_rate': 0.06687564153051903, 
 'reg_lambda': 5.988827008099877e-08, 
 'random_strength': 43.00737258055013, 
 'depth': 6, 
 'min_data_in_leaf': 11, 
 'leaf_estimation_iterations': 4,
 'border_count': 160}
#Best score: 12222.074711313806

In [ ]:
cat_params_1= {
             #'task_type' : 'GPU',
             #'devices' : '0' ,
             "random_state":0,
'iterations': 892,
'objective': 'MAE',
'od_wait': 559,
'bootstrap_type': 'Bayesian',
'learning_rate': 0.08235364801150621,
'reg_lambda': 1.376560113624677e-07,
'random_strength': 11.977636358612905, 
'depth': 3,
'min_data_in_leaf': 28, 
'leaf_estimation_iterations': 8, 
'border_count': 205,
'bagging_temperature': 3.4081517294621726}
#Best score: 12667.30812952728

In [ ]:
pred_oof = []
pred_tests = []
scores = []

kfold = KFold(n_splits=5, shuffle=True, random_state=0)

for fold, (idx_train, test_idx) in enumerate(kfold.split(df_train, y)):
    X_train, X_test = df_train.iloc[idx_train], df_train.iloc[test_idx]
    y_train, y_test = y.iloc[idx_train], y.iloc[test_idx]

    model = CatBoostRegressor(**cat_params, eval_metric='MAE')
    model.fit(X_train, y_train,
              eval_set = [(X_test, y_test)], early_stopping_rounds=20, verbose=0,
              use_best_model=True)
    
    plt.figure(figsize=(5,5))
    plt.plot(model.evals_result_["learn"]["MAE"], label="Training Correlation")
    plt.plot(model.evals_result_["validation"]["MAE"], label="Validation Correlation")
    plt.xlabel("Number of trees")
    plt.ylabel("Correlation")
    plt.legend()
    plt.show()

    pred_train = model.predict(X_train)
    pred_test = model.predict(df_test)
    mae = mean_absolute_error(y_train, pred_train)
    scores.append(mae)
    pred_oof.extend(pred_train)
    pred_tests.append(pred_test)
    
    print(f"Fold {fold}: MAE = {mae}")
    print("--"*20)
    
print(f"Overall MAE:{ np.mean(scores)}")


In [ ]:
sample_submission = pd.read_csv("../input/home-data-for-ml-course/sample_submission.csv")
predictions = np.mean(np.column_stack(pred_tests), axis=1)
sample_submission["SalePrice"] = predictions
sample_submission.to_csv("pred_cat_7.csv", index=False)
sample_submission
